In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[2],2
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,"[-2, -2, 0]"
ng_refs,-,"[2, 3, 6]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.150506                    1  0.077059
0.000750        2  0.150897                    2  0.078089
0.001052        3  0.154483                    3  0.081454
0.001476        4  0.157811                    4  0.084734
0.002070        5  0.160774                    5  0.087903
0.002904        6  0.163010                    6  0.090976
0.004074        7  0.164338                    7  0.093824
0.005714        8  0.164455                    8  0.096380
0.008015        9  0.163082                    9  0.098433
0.011243       10  0.159821                   10  0.099826
0.015771       11  0.154195                   11  0.100222
0.022122       12  0.145761                   12  0.099119
0.031031       13  0.133896                   13  0.095954
0.043528       14  0.118228                   14  0.090130
0.061057       15  0.107771                   15  0.088777
0.085645       16  0.118233                   16  0.105131
0.120136       17  0.142700                   17  0.133958
0.168516       18  0.163342                   18  0.158458
0.236378       19  0.179192                   19  0.174678
0.331549       20  0.192917                   20  0.184972
0.465100       21  0.209116                   21  0.199238
0.652400       22  0.235296                   22  0.232652
0.915100       23  0.259517                   23  0.269539
1.283650       24  0.231296                   24  0.248092
1.800600       25  0.173344                   25  0.186992
2.525700       26  0.134207                   26  0.139613
3.542800       27  0.110392                   27  0.111068
4.969550       28  0.094594                   28  0.093825
6.970850       29  0.083647                   29  0.082529
9.778100       30  0.075054                   30  0.071828
13.715850      31  0.066558                   31  0.062283
19.239350      32  0.057801                   32  0.054913
26.987250      33  0.051570                   33  0.050315
37.855300      34  0.048001                   34  0.046054
53.100050      35  0.043697                   35  0.040250
73.887500      36  0.038405                   36  0.035178
97.662500      37  0.035530                   37  0.033993
121.437500     38  0.034808                   38  0.034783
145.212500     39  0.032620                   39  0.033374
168.987500     40  0.032527                   40  0.033754
192.762500     41  0.057578                   41  0.059533
216.537500     42  0.167167                   42  0.170318
240.312500     43  0.326836                   43  0.315035
264.087500     44  0.500570                   44  0.466857
287.862500     45  0.661044                   45  0.626283
311.637500     46  0.695882                   46  0.637963
335.412500     47  0.710241                   47  0.637296
359.187500     48  0.739594                   48  0.687137
382.962500     49  0.778400                   49  0.744968
406.737500     50  0.822551                   50  0.788749
430.512500     51  0.866454                   51  0.830939
454.287500     52  0.888889                   52  0.866048
478.062500     53  0.901492                   53  0.901797
501.837500     54  0.910508                   54  0.928925
525.612500     55  0.914068                   55  0.940262
549.387500     56  0.911287                   56  0.934578
573.162500     57  0.902917                   57  0.922314
596.937500     58  0.923258                   58  0.947896
620.712500     59  0.920720                   59  0.962573
644.487500     60  0.890931                   60  0.956795
668.262500     61  0.840791                   61  0.922757
692.037500     62  0.776111                   62  0.853613
715.812500     63  0.703552                   63  0.757758
739.587500     64  0.628612                   64  0.650623
763.362500     65  0.555618                   65  0.549708
78

# Fluxes by Layer

CRD                           CLIRAD             \
                        flug       fldg      fnetg       flug       fldg   
pressure    level                                                          
0.000000    1     -60.366688   0.000000 -60.366688 -60.074834   0.000002   
0.000624    2     -60.366695   0.000017 -60.366677 -60.074841   0.000015   
0.000876    3     -60.366697   0.000024 -60.366673 -60.074843   0.000019   
0.001229    4     -60.366700   0.000034 -60.366666 -60.074847   0.000026   
0.001723    5     -60.366705   0.000048 -60.366657 -60.074852   0.000036   
0.002417    6     -60.366712   0.000068 -60.366644 -60.074859   0.000051   
0.003391    7     -60.366721   0.000096 -60.366625 -60.074869   0.000071   
0.004757    8     -60.366734   0.000136 -60.366598 -60.074882   0.000100   
0.006672    9     -60.366752   0.000191 -60.366561 -60.074901   0.000141   
0.009359    10    -60.366778   0.000269 -60.366509 -60.074928   0.000199   
0.013128    11    -60.366813   0.000375 -60.366438 -60.074965   0.000281   
0.018415    12    -60.366863   0.000522 -60.366341 -60.075018   0.000396   
0.025830    13    -60.366934   0.000721 -60.366213 -60.075092   0.000557   
0.036232    14    -60.367035   0.000987 -60.366048 -60.075198   0.000782   
0.050823    15    -60.367183   0.001339 -60.365844 -60.075351   0.001090   
0.071291    16    -60.367398   0.001816 -60.365583 -60.075570   0.001524   
0.100000    17    -60.367698   0.002518 -60.365181 -60.075873   0.002185   
0.140271    18    -60.368093   0.003594 -60.364500 -60.076271   0.003222   
0.196760    19    -60.368598   0.005190 -60.363407 -60.076776   0.004788   
0.275997    20    -60.369211   0.007485 -60.361725 -60.077386   0.007037   
0.387100    21    -60.369899   0.010712 -60.359187 -60.078061   0.010147   
0.543100    22    -60.370540   0.015218 -60.355323 -60.078672   0.014440   
0.761700    23    -60.370723   0.021494 -60.349229 -60.078788   0.020581   
1.068500    24    -60.369587   0.029790 -60.339797 -60.077522   0.029112   
1.498800    25    -60.366953   0.038946 -60.328008 -60.074710   0.038947   
2.102400    26    -60.363545   0.047930 -60.315614 -60.071080   0.048688   
2.949000    27    -60.359586   0.057432 -60.302154 -60.066863   0.058474   
4.136600    28    -60.355091   0.068467 -60.286624 -60.062050   0.069288   
5.802500    29    -60.350052   0.082095 -60.267958 -60.056639   0.082394   
8.139200    30    -60.344352   0.099548 -60.244804 -60.050478   0.099079   
11.417000   31    -60.337827   0.122164 -60.215663 -60.043403   0.119896   
16.014700   32    -60.330702   0.151289 -60.179413 -60.035647   0.146065   
22.464000   33    -60.323986   0.188730 -60.135255 -60.028331   0.180705   
31.510500   34    -60.318430   0.238439 -60.079992 -60.022311   0.228609   
44.200100   35    -60.314177   0.306339 -60.007839 -60.017820   0.293353   
62.000000   36    -60.313681   0.397979 -59.915703 -60.017575   0.377985   
85.775000   37    -60.322901   0.515358 -59.807542 -60.027943   0.487436   
109.550000  38    -60.340078   0.632599 -59.707479 -60.046933   0.602171   
133.325000  39    -60.362521   0.753074 -59.609447 -60.071791   0.724998   
157.100000  40    -60.392345   0.874765 -59.517579 -60.104432   0.851639   
180.875000  41    -60.432253   1.006281 -59.425973 -60.147006   0.989285   
204.650000  42    -60.492928   1.229112 -59.263815 -60.210776   1.220736   
228.425000  43    -60.610825   1.817804 -58.793020 -60.336053   1.825731   
252.200000  44    -60.802607   2.930060 -57.872546 -60.539589   2.916597   
275.975000  45    -61.086455   4.623674 -56.462782 -60.836778   4.528740   
299.750000  46    -61.482982   6.881911 -54.601070 -61.246614   6.702569   
323.525000  47    -61.951167   9.309923 -52.641245 -61.727022   8.979869   
347.300000  48    -62.471948  11.830968 -50.640979 -62.259644  11.307505   
371.075000  49    -63.042200  14.484151 -48.558049 -62.841638  13.824895   
394.850000  50    -63.658004  17.292176 -46.365827 -63.468188  16.549730 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')